In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
from extractor.forex_extractor import FOREXExtractor
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2023-06-13 2023-06-14


In [4]:
market.connect()
data = FOREXExtractor.extract(start,end)
values = pd.DataFrame(data["rates"].values.tolist())
dates = data["rates"].keys().tolist()
values["dates"]= dates
market.store("forex",values)
market.disconnect()

Expecting value: line 1 column 1 (char 0)


TypeError: 'NoneType' object is not subscriptable

In [5]:
values

NameError: name 'values' is not defined

In [6]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("stocks",data)
        except:
            print(ticker,"tiingo")
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

  1%|█▋                                                                                                                                                                         | 5/490 [00:03<05:33,  1.45it/s]

market stocks documents must be a non-empty list


 73%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                             | 358/490 [03:50<01:25,  1.55it/s]

market stocks documents must be a non-empty list


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                 | 441/490 [04:43<00:32,  1.51it/s]

market stocks documents must be a non-empty list


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [05:15<00:00,  1.56it/s]


In [7]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:35<00:00, 13.67it/s]


In [8]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
282,-0.603352,-0.177606,0.092308,0.163934,0.163934,LUMN,Lumen Technologies,Communication Services,Alternative Carriers
118,-0.328351,0.034360,0.143269,0.063856,0.063856,CMA,Comerica,Financials,Diversified Banks
333,0.641653,0.458427,0.255319,0.057577,0.057577,NCLH,Norwegian Cruise Line Holdings,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
205,0.268140,0.141712,0.091141,0.053119,0.053119,GNRC,Generac,Industrials,Electrical Components & Equipment
201,0.057278,-0.030120,0.122639,0.052882,0.052882,FCX,Freeport-McMoRan,Materials,Copper
275,-0.173338,0.122444,0.143855,0.052248,0.052248,LNC,Lincoln Financial,Financials,Multi-line Insurance
450,0.153377,0.054026,0.208671,0.049978,0.049978,URI,United Rentals,Industrials,Trading Companies & Distributors
17,0.068888,0.047704,0.170349,0.047129,0.047129,ALB,Albemarle Corporation,Materials,Specialty Chemicals
445,-0.059093,-0.194906,0.098878,0.044362,0.044362,ULTA,Ulta Beauty,Consumer Discretionary,Specialty Stores
412,0.178588,0.119839,0.179952,0.042785,0.042785,SWK,Stanley Black & Decker,Industrials,Industrial Machinery


In [9]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Information Technology,0.203568,0.054128,0.039582,0.009982,0.009982
Industrials,0.101434,0.041266,0.071531,0.014122,0.014122
Consumer Discretionary,0.131615,0.026662,0.077833,0.012056,0.012056
Communication Services,0.119989,0.021022,0.054901,0.013917,0.012078
Real Estate,0.003434,0.020986,0.055787,0.004066,0.004066
Financials,-0.050840,0.014961,0.054178,0.013143,0.013143
Health Care,0.005689,-0.002731,0.028239,0.006364,0.007538
Consumer Staples,0.011045,-0.009877,0.011971,0.004589,0.004589
Utilities,-0.039599,-0.015262,0.032552,0.000005,0.000005
